In [7]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetV2B2
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential


In [8]:
# 1. 클래스 수 정의
num_classes = 7
class_names = ['Andesite', 'Basalt', 'Etc', 'Gneiss', 'Granite', 'Mud_Sandstone', 'Weathered_Rock']

# 2. 모델 정의 및 가중치 로드
model = Sequential([
    EfficientNetV2B2(include_top=False, input_shape=(224, 224, 3), pooling='avg', weights='imagenet'),
    layers.Dropout(0.3),
    layers.Dense(num_classes, activation='softmax')
])
model.load_weights('./checkpoints/model_epoch_3.weights.h5')


In [9]:
# 3. test.csv 불러오기
test_df = pd.read_csv('open/test.csv')

# 4. 경로 보정 (img_path에 open/ 붙이기)
test_df['img_path'] = test_df['img_path'].apply(lambda x: os.path.join('open', x))

# 5. ImageDataGenerator로 배치 단위 이미지 로딩
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    x_col='img_path',
    y_col=None,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode=None,
    batch_size=32,
    shuffle=False
)


Found 95006 validated image filenames.


In [10]:
# 6. 예측 수행
preds = model.predict(test_generator, verbose=1)
pred_classes = np.argmax(preds, axis=1)
pred_labels = [class_names[i] for i in pred_classes]


C:\Users\FOR\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


2969/2969 ━━━━━━━━━━━━━━━━━━━━ 2799s 940ms/step


In [11]:
# 7. 결과 저장
submission = pd.DataFrame({
    'ID': test_df['ID'],
    'rock_type': pred_labels
})
submission.to_csv('submission_EfficientNetV2B2_epoch3.csv', index=False)
print("✅ 'submission_EfficientNetV2B2_epoch3.csv' 파일이 성공적으로 생성되었습니다.")


✅ 'submission_EfficientNetV2B2_epoch3.csv' 파일이 성공적으로 생성되었습니다.
